In [ ]:
import ee
import geemap
import numpy
import math
import sys
import time
sys.setrecursionlimit(30000)
ee.Authenticate()
ee.Initialize(project='yourself')

In [ ]:
################### Planting Date ########################
sowingYEAR = 2014
sowingYaD = sowingYEAR * 1000 + 250
harvestYaD = (sowingYEAR + 1) * 1000 + 190
start_year = str(sowingYaD)[:4]
sowway = 'projects/yourself/assets/new_sow' + start_year
sowdoy = ee.Image(sowway).subtract(250)


################### Bound information ########################
Map = geemap.Map()
vector = ee.FeatureCollection('projects/yourself/assets/WinterWheat_Region')
bound = vector.geometry().bounds()
proj = {
    'scale': 1000,
    'crs': 'EPSG:4326'}


################### Variety data ########################
variety = ee.Image('projects/yourself/assets/variety').resample('bilinear').reproject(
    scale = proj['scale'],
    crs = proj['crs']).clip(bound).subtract(1)
varietylist = ee.List.sequence(0,18,1)
IElist = ee.List([0.997, 0.987, 0.86, 0.995, 0.994, 0.987, 0.994, 0.981, 0.943, 0.994, 0.994, 0.956, 0.987, 0.962, 0.74, 0.879, 0.74, 0.835, 0.943])
PVTlist = ee.List([57.1, 56.2, 58.1, 59.7, 50.5, 13.3, 14.3, 51.4, 28.6, 56.2, 53.3, 43.8, 38.2, 13.3, 12.4, 26.7, 3.8, 5.7, 2.9])
PSlist = ee.List([0.000929, 0.000913, 0.00089, 0.000173, 0.000772, 0.000882, 0.001, 0.000394, 0.000961, 0.000575, 0.001, 0.000992,
                  0.000976, 0.000882, 0.00074, 0.000323, 0.000882, 0.000858, 0.000205])
TSlist = ee.List([1.04, 1.08, 1.25, 1.27, 1.29, 1.7, 1.69, 1.23, 1.31, 1.25, 1.46, 1.12, 1.21, 1.69, 1.02, 1.01, 1.04, 1.27, 1.65])
FDFlist = ee.List([0.935, 0.987, 0.996, 0.858, 0.916, 0.994, 0.948, 0.929, 0.813, 0.961, 0.994, 0.871, 0.987, 0.897, 0.839, 0.806, 0.8, 0.819, 0.8])
HIlist = ee.List([0.45, 0.43, 0.35, 0.39, 0.44, 0.45, 0.44, 0.45, 0.45, 0.43, 0.44, 0.38, 0.43, 0.49, 0.45, 0.35, 0.45, 0.38, 0.42])
SLAClist = ee.List([0.0025, 0.003, 0.0025, 0.003, 0.0028, 0.0035, 0.0032, 0.0032, 0.0022, 0.0022, 0.0025, 0.003, 0.0019, 0.0035, 0.0028, 0.0024, 0.0026, 0.0035, 0.0028])
def variety_function(varlist):
  def getvarvalue(value, varietyimg):
    listnum = ee.Number(value)
    Varietyimg = ee.Image(varietyimg).where(ee.Image(varietyimg).eq(listnum.add(1)), ee.Number(varlist.get(listnum)))
    return Varietyimg
  return ee.Image(varietylist.iterate(getvarvalue, variety))

IE = variety_function(IElist)
PVT = variety_function(PVTlist)
PS = variety_function(PSlist)
TS = variety_function(TSlist)
FDF = variety_function(FDFlist)
HI = variety_function(HIlist)
SLAC = variety_function(SLAClist)

In [ ]:
# Generate an empty image with latitude and longitude information
LATimage = ee.Image.pixelLonLat().clip(bound).select('latitude').rename('LAT').reproject(
        scale = proj['scale'],
        crs = proj['crs']).clip(bound)


################### Calculation of the reproductive period (PDT) / WheatDevelopmentModel ########################
########## Getting information on Julian calendar days #########
def GetDOY(sowingYaD, harvestYaD):
  start_year = int(str(sowingYaD)[:4])
  end_year = int(str(harvestYaD)[:4])
  if start_year % 4 == 0 and (start_year % 100 != 0 or start_year % 400 == 0):
    year_days = 366
  else:
    year_days = 365
  a = sowingYaD % 1000
  b = harvestYaD % 1000
  DOY1 = range(a, year_days + 1)
  DOY2 = range(1, b+1)
  DOY = ee.Array([doy for doy in DOY1] + [doy for doy in DOY2])
  return DOY


######################## Daily Deviation DEC ########################
def CalDEC(DOY):
  sin = ee.Array(math.sin(23.45 * math.pi / 180))
  cos = DOY.add(10).multiply((2 * math.pi)).divide(365).cos()
  SC = sin.multiply(cos)
  tempDEC = SC.multiply(-1).asin()
  return ee.List(tempDEC)


######################## Intermediate variable SSIN、CCOS、SSCC ########################
def CalSSIN(LATimage, DEC):
  LAT = LATimage.select('LAT')
  DEC = DEC.getInfo()
  SSIN_list = []
  i = 0
  while i < num:
    SSIN = LAT.expression(
        'sin(Lat * (pi / 180)) * sin(DEC)',{
            'Lat': LAT,
            'pi': math.pi,
            'DEC': DEC[i]
            }).rename('ssin')
    SSIN_list.append(SSIN)
    i += 1
  return ee.ImageCollection(SSIN_list)


def CalCCOS(LATimage, DEC):
  LAT = LATimage.select('LAT')
  DEC = DEC.getInfo()
  CCOS_list = []
  i = 0
  while i < num:
    CCOS = LAT.expression(
        'cos(Lat * (pi / 180)) * cos(DEC)',{
            'Lat': LAT,
            'pi': math.pi,
            'DEC': DEC[i]
            }).rename('ccos')
    CCOS_list.append(CCOS)
    i += 1
  return ee.ImageCollection(CCOS_list)


def CalSSCC(SSIN, CCOS):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calcsscc(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    sscc = ssin.divide(ccos)
    return sscc.rename('sscc')
  return merged_collection.map(calcsscc)


#################### Length of day DL ########################
def CalDL(SSCC):
  def caldl(sscc):
    DL = sscc.expression(
        '12 * (1 + 2 * asin(SSCC) / pi) + 40/60',{
            'SSCC': sscc,
            'pi': math.pi
            }).rename('dl')
    return DL
  return SSCC.map(caldl)


################### Calculation of relative photoperiodic effect RPE ########################
def CalRPE(PS, DL):
  def calrpe(dl):
    dayRPE = dl.expression(
        '1.0 - PS * pow(DL - 20, 2)',{
            'PS': PS,
            'DL': dl
            })
    return dayRPE
  return DL.map(calrpe)


############### Daily solar effective altitude DSINBE ########################
def CalDSINBE(DL, SSIN, CCOS, SSCC):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(SSCC.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(DL.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def caldsinbe(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    sscc = image.select('sscc')
    dl = image.select('dl')
    DSINBE_1 = ssin.expression(
        'SSIN + 0.4 * (pow(SSIN, 2) + pow(CCOS, 2) * 0.5)',{
            'SSIN': ssin,
            'CCOS': ccos
            })
    DSINBE_2 = ssin.expression(
        'CCOS * (2 + 3 * 0.4 * SSIN) * sqrt(1 - pow(SSCC, 2)) / pi',{
            'CCOS': ccos,
            'SSIN': ssin,
            'SSCC': sscc,
            'pi': math.pi
            })
    DSINBE = dl.expression(
        '3600 * (DL * DSINBE_1 + 12 * DSINBE_2)',{
            'DL': dl,
            'DSINBE_1': DSINBE_1,
            'DSINBE_2': DSINBE_2
            })
    return DSINBE.rename('dsinbe')
  return merged_collection.map(caldsinbe)


######################## Solar time Th_j ########################
def CalTh_j(DL):
  DIS3 = [0.112702, 0.5, 0.887298]
  def calth_j(dl):
    Th_j_bandlist = []
    j = 0
    while j < 3:
      Th_j_j = dl.expression(
          '12 + 0.5 * DL * DIS3',{
              'DL': dl,
              'DIS3': DIS3[j]
              }).rename('th_j_' + str(j))
      Th_j_bandlist.append(Th_j_j)
      j += 1
    Th_j_image = ee.Image.cat(Th_j_bandlist)
    return Th_j_image
  return DL.map(calth_j)


######################## Sine of solar altitude angle at the jth moment of day i SINB_ij ########################
def CalSINB_ij(SSIN, CCOS, Th_j):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(Th_j.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calsinb_ij(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    SINB_ij_bandlist = []
    j = 0
    while j < 3:
      th_j = image.select('th_j_' + str(j))
      SINB_ij_j = ssin.expression(
        'SSIN + CCOS * cos(2 * pi * (Th_j + 12) / 24)',{
            'SSIN': ssin,
            'CCOS': ccos,
            'pi': math.pi,
            'Th_j': th_j
            }).rename('sinb_ij_' + str(j))
      SINB_ij_bandlist.append(SINB_ij_j)
      j += 1
    SINB_ij_image = ee.Image.cat(SINB_ij_bandlist)
    return SINB_ij_image
  return merged_collection.map(calsinb_ij)


######################## The jth moment canopy extinction coefficient K_j ########################
def CalK_j(SINB_ij):
  Con = ee.Image(0.42).clip(bound)
  def calk_j(image):
    K_j_bandlist = []
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      K_j_j = Con.divide(sinb_ij).rename('k_j_' + str(j))
      K_j_bandlist.append(K_j_j)
      j += 1
    K_j_image = ee.Image.cat(K_j_bandlist)
    return K_j_image
  return SINB_ij.map(calk_j)


######################## Reflectance of light from the canopy at the jth moment p_j ########################
def Calp_j(SINB_ij):
  o = 0.2
  def calp_j(image):
    p_j_bandlist = []
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      p_j_j = sinb_ij.expression(
          '((1 - sqrt(1 - o)) / (1 + sqrt(1 - o))) * (2.0 / (1 + 1.6 * SINB_ij))',{
              'o': o,
              'SINB_ij': sinb_ij
              }).rename('p_j_' + str(j))
      p_j_bandlist.append(p_j_j)
      j += 1
    p_j_image = ee.Image.cat(p_j_bandlist)
    return p_j_image
  return SINB_ij.map(calp_j)


################### SolarRadiation ERA ########################
def CalERAQ(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  Q_list = []
  i = 0
  def resample_image(image):
    return image.resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs'])
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Radiation_colllection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select('surface_solar_radiation_downwards_hourly')
    Radiation_colllection = Radiation_colllection.map(resample_image)
    dailyQ = Radiation_colllection.reduce(ee.Reducer.sum()).multiply(0.522).clip(bound).rename('q')
    Q_list.append(dailyQ)
    start_date = end_date
    i += 1
  return ee.ImageCollection(Q_list)


######################## Photosynthetic Active Radiation PAR_i ########################
def CalPAR_i(Q, SINB_ij, DSINBE):
  merged_collection = DSINBE.map(lambda image: image.addBands(SINB_ij.filterMetadata('system:index', 'equals', image.get('system:index')).first())) \
                 .map(lambda image: image.addBands(Q.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calpar_i(image):
    PAR_i_bandlist = []
    dsinbe = image.select('dsinbe')
    q = image.select('q')
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      PAR_i_j = sinb_ij.expression(
          '0.5 * Q * SINB_ij * (1.0 + 0.4 * SINB_ij) / DSINBE',{
              'Q': q,
              'SINB_ij': sinb_ij,
              'DSINBE': dsinbe
          }).rename('par_i_' + str(j))
      PAR_i_bandlist.append(PAR_i_j)
      j += 1
    PAR_i_image = ee.Image.cat(PAR_i_bandlist)
    return PAR_i_image
  return merged_collection.map(calpar_i)


################### Calculate the average temperature T24H ########################
def CalT24H(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  T24H_list = []
  i = 0
  def kelvin_to_celsius(image):
    return image.subtract(273.15).resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs'])
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Temperature_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select(['temperature_2m'])
    Temperature_collection = Temperature_collection.map(kelvin_to_celsius)
    dailyT = Temperature_collection.reduce(ee.Reducer.mean()).clip(bound)
    T24H_list.append(dailyT)
    start_date = end_date
    i += 1
  return ee.ImageCollection(T24H_list)


################### Temperature per time period Ti ########################
def CalTi(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  Ti_list = []
  def kelvin_to_celsius(image):
    return image.subtract(273.15).resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs']).clip(bound)
  i = 0
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Temperature_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select(['temperature_2m'])
    Temperature_collection = Temperature_collection.map(kelvin_to_celsius)
    Ti_image = Temperature_collection.toBands()
    Ti_list.append(Ti_image)
    start_date = end_date
    i += 1
  return ee.ImageCollection(Ti_list)


################### Daily Tmax、Tmin ########################
def GetdayTmax(Ti):
  def caltmax(image):
    tmax = image.reduce(ee.Reducer.max())
    return tmax.rename('tmax_d')
  return Ti.map(caltmax)

def GetdayTmin(Ti):
  def caltmin(image):
    tmin = image.reduce(ee.Reducer.min())
    return tmin.rename('tmin_d')
  return Ti.map(caltmin)


####################### Calculate the average daily temperature ########################
def CaldTmean(Tmax, Tmin):
  merged_collection = Tmax.map(lambda image: image.addBands(Tmin.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def caldtmean(image):
    tmax = image.select('tmax_d')
    tmin = image.select('tmin_d')
    dT = tmax.multiply(0.6).add(tmin.multiply(0.4))
    return dT
  return merged_collection.map(caldtmean)


################### Calculation of daily vernalization effect DVE ########################
def CalDVE(PVT, Ti):
  vef = ee.Image(1).divide(ee.Image(2).subtract(ee.Image(0.0167).multiply(PVT))).clip(bound)
  Tou = ee.Image(10).subtract(PVT.divide(20)).clip(bound)
  Tmv = ee.Image(18).subtract(PVT.divide(8)).clip(bound)
  Tbv = -1
  Tol = 1
  def caldve(dayT):
    dayDVE = ee.Image(0).clip(bound)
    j = 0
    while j < 9:
      DVEI = dayT.expression(
          '((Tbv <= dayT) && (dayT < Tol)) ? pow(sin(((dayT - Tbv) * pi / ((Tol - Tbv) * 2))), 0.5) ' +
          ': ((Tol <= dayT) && (dayT < Tou)) ? 1 ' +
          ': ((Tou <= dayT) && (dayT <= Tmv)) ? cos((dayT - Tou) * pi / ((Tmv - Tou) * 2)) > 0 ? \
          pow(cos((dayT - Tou) * pi / ((Tmv - Tou) * 2)), vef) : pow(0.122887278627, vef) ' +
          ': 0 ' ,{
              'Tbv': Tbv,
              'dayT': dayT.select(j),
              'Tol': Tol,
              'pi': math.pi,
              'Tou': Tou,
              'Tmv': Tmv,
              'vef': vef
              })
      dayDVE = dayDVE.add(DVEI)
      j += 1
    tem_DVE = dayDVE.divide(9)
    return tem_DVE
  return Ti.map(caldve)


################### Daily devernalization effect DDEVE ########################
def CalDDEVE(Ti):
  def calddeve(ti):
    dayDDEVE = ee.Image(0).clip(bound)
    j = 0
    while j < 9:
      DEVEI = ti.expression(
          'dayT > 27 ? (dayT - 27) * 0.5 : 0',{
          'dayT': ti.select(j)
      })
      dayDDEVE = dayDDEVE.add(DEVEI)
      j += 1
    tem_DDEVE = dayDDEVE.divide(9)
    return tem_DDEVE
  return Ti.map(calddeve)


################### Vernalization effect VD ########################
def CalVD(sowdoy, PVT, DDEVE, DVE):
  VD_list = []
  tempPVT = PVT.multiply(0.3)
  sumVD = ee.Image(0).clip(bound)
  i = 0
  while i < num:
    dve = DVE.filterMetadata('system:index', 'equals', str(i)).first()
    ddeve = DDEVE.filterMetadata('system:index', 'equals', str(i)).first()
    sumVD = sumVD.expression(
        'i <= sowdoy ? 0 ' +
        ': ((0 <= sumVD) && (sumVD <= tempPVT)) ? (DVE - DDEVE) > 0 ? (sumVD + DVE - DDEVE) : sumVD' +
        ': ((tempPVT < sumVD) && (sumVD <= PVT)) ? (sumVD + DVE) : sumVD',{
            'i': i,
            'sowdoy': sowdoy,
            'sumVD': sumVD,
            'tempPVT': tempPVT,
            'DDEVE': ddeve,
            'PVT': PVT,
            'DVE': dve
            }).clip(bound)
    VD_list.append(sumVD)
    i += 1
  return ee.ImageCollection(VD_list)


################# Vernalization process VP ########################
def CalVP(VD, PVT):
  def calvp(vd):
    dayVP = vd.expression(
        '(VD / PVT) <= 1 ? VD / PVT : 1',{
            'VD': vd,
            'PVT': PVT
        })
    return dayVP.clip(bound)
  return VD.map(calvp)


#################### Factors affecting the maximum rate of photosynthesis by mean daily temperature FTMP #######################
def CalFTMP(T24H):
  Tb = 0
  Tol = 15
  Tou = 25
  Tmax = 45
  def calftmp(t24h):
    tempFTMP = t24h.expression(
        '(Tb <= T24H) && (T24H < Tol) ? sin(((T24H - Tb) / (Tol - Tb)) * pi / 2) ' +
        ': (Tol <= T24H) && (T24H < Tou) ? 1 ' +
        ': (Tou <= T24H) && (T24H < Tmax) ? cos(((T24H - Tou) / (Tmax - Tou)) * pi / 2) ' +
        ': 0.001 ',{
            'Tb': Tb,
            'T24H': t24h,
            'Tol': Tol,
            'pi': math.pi,
            'Tou': Tou,
            'Tmax': Tmax
        })
    return tempFTMP
  return T24H.map(calftmp)


################### GDD, PDT, FA, PIS, PILVG, PISP, SLA, AMAX, LwpCr ########################
def CalPDT(PlantingDepth, RPE, VP, PVT, VD, FDF, Ti, TS, IE, sowdoy, FTMP, FCO2):
  DayGDD_list = []
  PDT_list = []
  FA_list = []
  PIS_list = []
  PILV_list = []
  PISP_list = []
  SLA_list = []
  AMAX_list = []
  LwpCr_list = []
  PDTTS = 16.1
  PDTHD = 26.8
  AMX = ee.Image(40).clip(bound)
  EMGDD = 60 + 30 * PlantingDepth
  DPE = ee.Image(0).clip(bound)
  tempPDT = ee.Image(0).clip(bound)
  TempSumDTT = ee.Image(0).clip(bound)
  daySLA = ee.Image(0.002171).clip(bound)
  i = 0
  while i < num:
    Tb = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 0 : tempPDT < 26.8 ? 3.3 : tempPDT < 39 ? 8 : 8',{
            'tempPDT': tempPDT})
    To = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 20 : tempPDT < 26.8 ? 22 : tempPDT < 39 ? 25 : 25',{
            'tempPDT': tempPDT})
    Tm = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 32 : tempPDT < 26.8 ? 32 : tempPDT < 39 ? 35 : 35',{
            'tempPDT': tempPDT})
    dayGDD = ee.Image(0).clip(bound)
    dayT = Ti.filterMetadata('system:index', 'equals', str(i)).first()
    SumTE_i = ee.Image(0).clip(bound)
    vd = VD.filterMetadata('system:index', 'equals', str(i)).first()
    rpe = RPE.filterMetadata('system:index', 'equals', str(i)).first()
    vp = VP.filterMetadata('system:index', 'equals', str(i)).first()
    ftmp = FTMP.filterMetadata('system:index', 'equals', str(i)).first()
    for j in range(9):
      RTEI = Tb.expression(
          '((Tb <= dayT) && (dayT <= To)) ? pow(sin((dayT - Tb) * pi / (2 * (To - Tb))), TS)' +
          ': ((To <= dayT) && (dayT <= Tm)) ? tempPDT < 26.8 ? pow(cos(pow((dayT - To) * pi / ((Tm - To) * 2), (Tm - To) / (To - Tb))), TS)' +
          ': pow(sin((Tm - dayT) * pi / ((Tm - To) * 2)), ((Tm - To) / (To - Tb)) * TS)' +
          ': 0 ',{
              'Tb': Tb,
              'dayT': dayT.select(j),
              'To': To,
              'pi': math.pi,
              'TS': TS,
              'Tm': Tm,
              'tempPDT': tempPDT
          })
      SumTE_i = SumTE_i.add(RTEI)
      TempSumDTT = TempSumDTT.expression(
            'i >= sowdoy ? dayT >= Tb ? TempSumDTT + (dayT - Tb) : TempSumDTT : 0',{
                'i': i,
                'sowdoy': sowdoy,
                'dayT': dayT.select(j),
                'Tb': Tb,
                'TempSumDTT': TempSumDTT
            })
      dayGDD = dayGDD.expression(
        'dayT >= Tb ? dayGDD + (dayT - Tb) : dayGDD ',{
            'dayT': dayT.select(j),
            'Tb': Tb,
            'dayGDD': dayGDD
        })
    DTE = SumTE_i.divide(9).clip(bound)
    GDD = TempSumDTT.divide(9).clip(bound)

    ######################## Daily GDD ########################
    DayGDD = dayGDD.divide(9).clip(bound)
    DayGDD_list.append(DayGDD)
    DTS = vd.expression(
        'VD < PVT ? RPE * VP : ((VD >= PVT) && (tempPDT <= PDTTS)) ? RPE : ((PDTTS < tempPDT) && (tempPDT < PDTHD)) ? ' +
        ' RPE + (1 - RPE) * (tempPDT - PDTTS) / (PDTHD - PDTTS)' +
        ': 1 ',{
            'VD': vd,
            'PVT': PVT,
            'RPE': rpe,
            'VP': vp,
            'tempPDT': tempPDT,
            'PDTTS': PDTTS,
            'PDTHD': PDTHD
        })
    DPE = GDD.expression(
        'GDD >= EMGDD ? DTE * DTS * IE * 0.6 : 0',{
            'GDD': GDD,
            'EMGDD': EMGDD,
            'DTE': DTE,
            'DTS': DTS,
            'IE': IE
        })
    tempPDT = tempPDT.expression(
        'tempPDT < 26.8 ? tempPDT + DPE : tempPDT < 39 ? tempPDT + DTE : tempPDT < 56 ? tempPDT + DTE * FDF : 56',{
            'tempPDT': tempPDT,
            'DPE': DPE,
            'DTE': DTE,
            'FDF': FDF
        }).clip(bound)
    PDT_list.append(tempPDT)

    ################### Factors influencing the maximum rate of photosynthesis by physiological age FA ########################
    tempFA = tempPDT.expression(
        'PDT >= 31 ? 1 - 0.02 * (PDT - 31) : 1',{
            'PDT': tempPDT,
        }).clip(bound)
    FA_list.append(tempFA)

    ######################## Ground allocation index PIS ########################
    tempPIS = tempPDT.expression(
    'PDT > 0.2 ? 0.8819 / (1 + 1.215 * exp(-0.0476 * PDT)) : 0',{
        'e': math.e,
        'PDT': tempPDT
    }).clip(bound)
    PIS_list.append(tempPIS)

    ######################## Green leaf allocation index PILV ########################
    tempPILV = tempPDT.expression(
    'PDT <= 56 ? 0.809 / (1 + 51.4 * exp(-0.112 * (56 - PDT))) : 0',{
        'PDT': tempPDT,
        'e': math.e
    }).clip(bound)
    PILV_list.append(tempPILV)

    ######################## Spike allocation index PISP ########################
    tempPISP = tempPDT.expression(
        'PDT >= 7.4 ? (0.11 + HI) * pow(sin((PDT - 7.4) * pi / (46.6 * 2)), (4 + HI)) : 0',{
            'PDT': tempPDT,
            'HI': HI,
            'pi': math.pi
    }).clip(bound)
    PISP_list.append(tempPISP)

    ######################## Specific leaf area SLA ########################
    DAYSLA = daySLA
    daySLA = tempPDT.expression(
        'PDT < 56 ? SLAc * exp(-0.73 / (56 - PDT)) : SLAc * exp(-0.73)',{
            'PDT': tempPDT,
            'SLAc': SLAC
        })
    SLA = daySLA.expression(
      '(PDT > 55) && (SLA_Y < SLA) ? SLA_Y : SLA',{
          'PDT': tempPDT,
          'SLA_Y': DAYSLA,
          'SLA': daySLA
      }).clip(bound)
    SLA_list.append(SLA)

    ######################## Maximum photosynthesis rate of a single leaf AMAX ########################
    tempAMAX = AMX.multiply(ftmp).multiply(tempFA).multiply(FCO2).clip(bound).rename('amax')
    AMAX_list.append(tempAMAX)

    ######################## Early morning critical leaf water potential LwpCr ########################
    tempLwpCr = ee.Image(0).expression(
        'PDT < 16.1 ? -10 ' +
         ': (16.1 <= PDT) && (PDT < 21.4) ? -8 ' +
         ': (21.4 <= PDT) && (PDT < 31) ? -9 ' +
         ': -12',{
             'PDT':tempPDT})
    LwpCr_list.append(tempLwpCr)
    i += 1
  return ee.ImageCollection(DayGDD_list), ee.ImageCollection(PDT_list), ee.ImageCollection(FA_list), ee.ImageCollection(PIS_list), ee.ImageCollection(PILV_list), \
      ee.ImageCollection(PISP_list), ee.ImageCollection(SLA_list), ee.ImageCollection(AMAX_list), ee.ImageCollection(LwpCr_list)


############## PDT Analog Module (Solar Temperature Module) ##############

SSIN = CalSSIN(LATimage, DEC)
CCOS = CalCCOS(LATimage, DEC)
SSCC = CalSSCC(SSIN, CCOS)
DL = CalDL(SSCC)
RPE = CalRPE(PS, DL)
DSINBE = CalDSINBE(DL, SSIN, CCOS, SSCC)
Th_j = CalTh_j(DL)
SINB_ij = CalSINB_ij(SSIN, CCOS, Th_j)
K_j = CalK_j(SINB_ij)
p_j = Calp_j(SINB_ij)
Q = CalERAQ(sowingYaD)
PAR_i = CalPAR_i(Q, SINB_ij, DSINBE)
T24H = CalT24H(sowingYaD)
Ti = CalTi(sowingYaD)
Tmax = GetdayTmax(Ti)
Tmin = GetdayTmin(Ti)
dTmean = CaldTmean(Tmax, Tmin)
DVE = CalDVE(PVT, Ti)
DDEVE = CalDDEVE(Ti)
VD = CalVD(sowdoy, PVT, DDEVE, DVE)
VP = CalVP(VD, PVT)
FTMP = CalFTMP(T24H)
FCO2 = ee.Image(1 + 0.8 * numpy.log(414.02/340)).clip(bound)
GDD, PDT, FA, PIS, PILVG, PISP, SLA, AMAX, LwpCr = CalPDT(PlantingDepth, RPE, VP, PVT, VD, FDF, Ti, TS, IE, sowdoy, FTMP, FCO2)